In [7]:
import math
import numpy as np
from numba import cuda, jit
from datetime import datetime

CPU_tmp = 0 
GPU_tmp = 0 
GPU_time = []
GPU_matsize = []

def info(TBS): 
  A = np.random.uniform (-100, 100, TBS*TBS*TBS*TBS)
  A = A.reshape(TBS*TBS, TBS*TBS)
  B = np.random.uniform (-100, 100, TBS*TBS*TBS*TBS)
  B = B.reshape(TBS*TBS, TBS*TBS)
  C = np.zeros((TBS*TBS,TBS*TBS ))

  @cuda.jit 
  def MatMul_cuda(A, B, C):
    i, j = cuda.grid(2)
    if i < C.shape[0] and j < C.shape[1]:
      sum = 0
      for k in range(A.shape[1]):
        sum += A[i, k] * B[k, j]
        C[i, j] = sum

      
  def host_naive(A,B,C): 
    d_A = cuda.to_device(A) 
    d_B = cuda.to_device(B)
    d_C = cuda.device_array(C.shape, np.float64)

    block = (TBS, TBS)
    blockgrid_x = math.ceil(A.shape[0]/ block[0])
    blockgrid_y = math.ceil(B.shape[1]/ block[1])
    blockgrid = (blockgrid_x, blockgrid_y)
    print(block)
    

    MatMul_cuda[blockgrid, block](d_A, d_B, d_C)

    return d_C.copy_to_host()

  start = datetime.now()
  host_naive(A,B,C)
  GPU_res = C
  print("GPU MatMul's time: ", datetime.now() - start)
  GPU_tmp=(datetime.now()-start)
  GPU_time.append(GPU_tmp.total_seconds())
  GPU_matsize.append(TBS*TBS*TBS*TBS)

TBS = 0
while TBS <=64:
  count = 1
  print()
  print("TBS", TBS)
  TBS +=1
  while count <=3:
    print("Cycle", count)
    info(TBS)
    count +=1
print ("Max size for TPB is 32!")


TBS 0
Cycle 1
(1, 1)
GPU MatMul's time:  0:00:00.333506
Cycle 2
(1, 1)
GPU MatMul's time:  0:00:00.246452
Cycle 3
(1, 1)
GPU MatMul's time:  0:00:00.240610

TBS 1
Cycle 1
(2, 2)
GPU MatMul's time:  0:00:00.237090
Cycle 2
(2, 2)
GPU MatMul's time:  0:00:00.240304
Cycle 3
(2, 2)
GPU MatMul's time:  0:00:00.250619

TBS 2
Cycle 1
(3, 3)
GPU MatMul's time:  0:00:00.229944
Cycle 2
(3, 3)
GPU MatMul's time:  0:00:00.232396
Cycle 3
(3, 3)
GPU MatMul's time:  0:00:00.233959

TBS 3
Cycle 1
(4, 4)
GPU MatMul's time:  0:00:00.237569
Cycle 2
(4, 4)
GPU MatMul's time:  0:00:00.236889
Cycle 3
(4, 4)
GPU MatMul's time:  0:00:00.231067

TBS 4
Cycle 1
(5, 5)
GPU MatMul's time:  0:00:00.233932
Cycle 2
(5, 5)
GPU MatMul's time:  0:00:00.332066
Cycle 3
(5, 5)
GPU MatMul's time:  0:00:00.228403

TBS 5
Cycle 1
(6, 6)
GPU MatMul's time:  0:00:00.233548
Cycle 2
(6, 6)
GPU MatMul's time:  0:00:00.233999
Cycle 3
(6, 6)
GPU MatMul's time:  0:00:00.237086

TBS 6
Cycle 1
(7, 7)
GPU MatMul's time:  0:00:00.231527
C

CudaAPIError: ignored